# 🚀 PET Resource Allocation Dashboard - Google Colab Deployment

This notebook will set up and run your PET Resource Allocation Dashboard in Google Colab with public access via ngrok.

## 📋 Instructions:
1. **Run all cells in order** (Runtime → Run all)
2. **Upload your CSV data** when prompted
3. **Get public URL** from the final cell
4. **Share the URL** with your team

---


In [ ]:
# 📦 STEP 1: Install Dependencies
print("🔧 Installing required packages...")

!pip install -q streamlit pandas plotly openpyxl pyngrok python-dateutil numpy watchdog

print("✅ Dependencies installed successfully!")


In [ ]:
# 📁 STEP 2: Create Directory Structure
print("📁 Setting up project structure...")

import os
os.makedirs('src/components', exist_ok=True)
os.makedirs('src/views', exist_ok=True)
os.makedirs('data', exist_ok=True)

print("✅ Directory structure created!")


In [ ]:
# 🔧 STEP 3: Create Core Schema Module
print("📝 Creating schema module...")

schema_code = '''"""
Schema definitions and column mappings for PET Resource Allocation Dashboard
"""

import re

# Sentinel values for header detection
SENTINELS = {"Supervisor or Hiring Manager", "Resource or Rec/Offer", "L3 Org"}

# Column mappings from business headers to normalized names
HEADER_MAP = {
    "Supervisor or Hiring Manager": "manager",
    "Resource or Rec/Offer": "resource_raw",
    "Type": "type",
    "L3 Org": "l3_org",
    "VP Org": "vp_org",
    "Director Org": "director_org",
    "Total Workstream Allocation %": "total_workstream_allocation_pct",
}

# Dynamic workstream detection patterns
WS_NAME_RE = re.compile(r"^\\s*Workstream\\s*(\\d+)\\s*$", re.I)
PCT_NAME_RE = re.compile(r"^\\s*%+\\s*(\\d+)\\s*$", re.I)

# Type normalization mapping
TYPE_MAPPING = {
    'employee': 'Employee',
    'contractor': 'Contractor',
    'req': 'Req',
    'open role': 'Open Role',
    'request': 'Req',
    'requisition': 'Req',
}

# Allocation status thresholds
ALLOCATION_THRESHOLDS = {
    'underallocated': 100,
    'overallocated': 100,
    'unassigned': 0
}

ALLOCATION_TOLERANCE = 1.0
WATCH_PATTERN = r'PET Resource Allocation.*\\.csv'
'''

with open('src/schema.py', 'w') as f:
    f.write(schema_code)

print("✅ Schema module created!")


In [ ]:
# 📊 STEP 4: Create ETL Module (Data Processing)
print("📊 Creating ETL module...")

etl_code = '''"""
ETL pipeline for PET Resource Allocation data
"""

import pandas as pd
import numpy as np
import re
from typing import Optional, Tuple
from .schema import HEADER_MAP, SENTINELS, WS_NAME_RE, PCT_NAME_RE, TYPE_MAPPING, ALLOCATION_TOLERANCE

def _likely_header_row(row_vals) -> bool:
    """Check if a row likely contains header information"""
    vals = {str(v).strip() for v in row_vals if pd.notna(v)}
    # require at least 2 sentinel hits to avoid false positives
    return len(SENTINELS.intersection(vals)) >= 2

def read_with_embedded_header(csv_path: str) -> pd.DataFrame:
    """Read CSV with embedded header detection (handles both embedded and standard formats)"""
    # 1) read raw with no header first
    raw = pd.read_csv(csv_path, header=None, dtype=str, keep_default_na=False)
    
    # 2) Check if first row has business headers (normal CSV)
    if len(raw) > 0 and _likely_header_row(raw.iloc[0].tolist()):
        # Standard format - headers in first row
        df = pd.read_csv(csv_path, header=0, dtype=str, keep_default_na=False)
        return df
    
    # 3) Look for embedded headers in first 10 rows
    header_idx = None
    scan_rows = min(10, len(raw))
    for i in range(scan_rows):
        if _likely_header_row(raw.iloc[i].tolist()):
            header_idx = i
            break
    
    # 4) For files with embedded headers, use the ORIGINAL column structure
    # but skip the business header row and any empty rows
    if header_idx is not None:
        # Use original pandas column names, skip to data after business headers
        df = pd.read_csv(csv_path, header=0, dtype=str, keep_default_na=False)
        # Skip the business header row (it becomes the first data row)
        if len(df) > header_idx:
            df = df.iloc[header_idx:].copy()
            # Remove the business header row itself
            if len(df) > 0:
                df = df.iloc[1:].copy()
        return df
    
    # 5) Fallback: try reading as normal CSV 
    df = pd.read_csv(csv_path, header=0, dtype=str, keep_default_na=False)
    return df

def find_workstream_pairs(cols):
    """Dynamically find workstream and percentage column pairs"""
    # Pattern 1: Look for "Workstream N" and "% N" pairs (legacy format)
    ws_cols = {}
    pct_cols = {}
    for c in cols:
        m1 = WS_NAME_RE.match(str(c))
        m2 = PCT_NAME_RE.match(str(c))
        if m1:
            ws_cols[int(m1.group(1))] = c
        elif m2:
            pct_cols[int(m2.group(1))] = c
    
    # Pattern 2: Look for "Workstream N" and "Workstream N.2" pairs (new format)
    if not pct_cols:  # If no % columns found, try the new format
        ws_name_pattern = re.compile(r"^\\s*Workstream\\s*(\\d+)\\s*$", re.I)
        ws_pct_pattern = re.compile(r"^\\s*Workstream\\s*(\\d+)\\.2\\s*$", re.I)
        
        for c in cols:
            m1 = ws_name_pattern.match(str(c))
            m2 = ws_pct_pattern.match(str(c))
            if m1:
                ws_cols[int(m1.group(1))] = c
            elif m2:
                pct_cols[int(m2.group(1))] = c
    
    pairs = []
    for k in sorted(set(ws_cols) & set(pct_cols)):
        pairs.append((ws_cols[k], pct_cols[k]))
    return pairs

def parse_employee(raw):
    """Parse employee information from raw resource string"""
    # "72633: Senior Software Engineer - Jane Doe" or similar
    if not isinstance(raw, str): 
        return None, None, None
    parts = raw.split(':', 1)
    if len(parts) == 2:
        emp_id = re.sub(r'\\D', '', parts[0]) or None
        title = parts[1].strip()
        name = title.split(' - ')[-1].strip() if ' - ' in title else title
        return emp_id, name, title
    return None, raw.strip(), raw.strip()

def compute_total_allocation(df):
    """Compute total allocation with tolerance for missing Total column"""
    pairs = find_workstream_pairs(df.columns)
    if not pairs:
        df["computed_total_pct"] = 0.0
        df["total_allocation_pct"] = df.get("total_workstream_allocation_pct", pd.Series([None]*len(df)))
        return df

    comp = None
    for _, pct_col in pairs:
        col = pd.to_numeric(df[pct_col], errors="coerce")
        if col.dropna().between(0,1).mean() > 0.85:  # scale
            col = col * 100.0
        comp = col if comp is None else comp.add(col, fill_value=0)
    df["computed_total_pct"] = comp.fillna(0)
    canon = "Total Workstream Allocation %"
    if canon in df.columns:
        df["total_allocation_pct"] = pd.to_numeric(df[canon], errors="coerce").fillna(df["computed_total_pct"])
    else:
        df["total_allocation_pct"] = df["computed_total_pct"]
    return df

def load_latest_csv(csv_path: str) -> pd.DataFrame:
    """End-to-end CSV loader with header detection"""
    df = read_with_embedded_header(csv_path)
    # rename canonical business columns to code-friendly names
    rename_map = {
        "Supervisor or Hiring Manager": "manager",
        "Resource or Rec/Offer": "resource_raw",
        "Type": "type",
        "L3 Org": "l3_org",
        "VP Org": "vp_org",
        "Director Org": "director_org",
        "Total Workstream Allocation %": "total_workstream_allocation_pct",
    }
    for k,v in rename_map.items():
        if k in df.columns:
            df = df.rename(columns={k: v})
        else:
            df[v] = ""

    # parse resource fields
    parsed = df["resource_raw"].apply(parse_employee)
    df[["employee_id","resource_name","resource_title"]] = pd.DataFrame(parsed.tolist(), index=df.index)

    # totals
    df = compute_total_allocation(df)

    # flags
    df["overallocated"] = df["total_allocation_pct"] > 100.0
    df["underallocated"] = df["total_allocation_pct"] < 100.0
    df["unassigned"] = df["total_allocation_pct"].fillna(0) == 0.0
    return df

def unpivot_assignments(df: pd.DataFrame) -> pd.DataFrame:
    """Create workstream assignments DataFrame"""
    pairs = find_workstream_pairs(df.columns)
    rows = []
    base_cols = ["resource_raw","type","manager","director_org","vp_org","l3_org"]
    # Add resource_name if available
    if "resource_name" in df.columns:
        base_cols.append("resource_name")
    
    for ws_col, pct_col in pairs:
        part = df[base_cols + [ws_col, pct_col]].copy()
        part = part.rename(columns={ws_col: "workstream", pct_col: "allocation_pct"})
        # numeric & scaling
        part["allocation_pct"] = pd.to_numeric(part["allocation_pct"], errors="coerce")
        # auto-scale if mostly 0–1
        if part["allocation_pct"].dropna().between(0,1).mean() > 0.85:
            part["allocation_pct"] = part["allocation_pct"] * 100.0
        part = part[part["allocation_pct"].fillna(0) > 0]
        rows.append(part)
    return pd.concat(rows, ignore_index=True) if rows else pd.DataFrame(columns=["workstream","allocation_pct"])

def process_pet_csv(file_path: str):
    """Main function to process PET CSV file"""
    try:
        # Use the new header-aware loader
        df = load_latest_csv(file_path)
        
        # Create workstream assignments using the new unpivot function
        workstream_df = unpivot_assignments(df)
        
        print(f"Processed {len(df)} resources with {len(workstream_df)} workstream assignments")
        
        return df, workstream_df
        
    except Exception as e:
        print(f"Error processing CSV {file_path}: {str(e)}")
        raise
'''

with open('src/etl.py', 'w') as f:
    f.write(etl_code)

print("✅ ETL module created!")


In [ ]:
# 🗃️ STEP 5: Create Store and Component Modules
print("🗃️ Creating remaining modules...")

# Create __init__.py files
with open('src/__init__.py', 'w') as f:
    f.write('')

with open('src/components/__init__.py', 'w') as f:
    f.write('')

with open('src/views/__init__.py', 'w') as f:
    f.write('')

# Create simplified store module
store_code = '''"""
Data storage and file handling
"""

import pandas as pd
from pathlib import Path
from .etl import process_pet_csv

def load_latest_data(data_dir="data"):
    """Load the most recent PET CSV file"""
    data_path = Path(data_dir)
    
    # Find CSV files matching pattern
    csv_files = list(data_path.glob("*PET*Resource*Allocation*.csv"))
    if not csv_files:
        csv_files = list(data_path.glob("*.csv"))
    
    if not csv_files:
        return None, None
    
    # Get most recent file
    latest_file = max(csv_files, key=lambda f: f.stat().st_mtime)
    
    try:
        df, workstream_df = process_pet_csv(str(latest_file))
        return df, workstream_df
    except Exception as e:
        print(f"Error loading data: {e}")
        return None, None

def get_data_store():
    """Simple data store getter"""
    return {"data_dir": "data"}
'''

with open('src/store.py', 'w') as f:
    f.write(store_code)

print("✅ Store module created!")


In [ ]:
# 📁 STEP 6: Upload Your Data Files
print("📁 Upload your PET Resource Allocation CSV files...")

from google.colab import files
import os

print("Please upload your CSV files using the file browser:")
print("1. Click the folder icon on the left sidebar")
print("2. Navigate to the 'data' folder")
print("3. Upload your PET Resource Allocation CSV files")
print("")
print("Or use the upload button below:")

# File upload widget
uploaded = files.upload()

# Move uploaded files to data directory
for filename, content in uploaded.items():
    if filename.endswith('.csv'):
        with open(f'data/{filename}', 'wb') as f:
            f.write(content)
        print(f"✅ Uploaded: {filename}")

print("✅ Data upload completed!")


In [ ]:
# 🎛️ STEP 7: Create Streamlit Application
print("🎛️ Creating Streamlit application...")

app_code = '''"""
PET Resource Allocation Dashboard - Streamlit Application
"""

import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

# Import our modules
from src.store import load_latest_data, get_data_store

# Page configuration
st.set_page_config(
    page_title="PET Resource Allocation Dashboard",
    page_icon="🏢",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS
st.markdown("""
<style>
    .stMetric {
        background-color: #f0f2f6;
        padding: 10px;
        border-radius: 5px;
    }
    .main-header {
        background: linear-gradient(90deg, #1f77b4, #ff7f0e);
        color: white;
        padding: 1rem;
        border-radius: 10px;
        text-align: center;
        margin-bottom: 2rem;
    }
</style>
""", unsafe_allow_html=True)

def main():
    # Header
    st.markdown("""
    <div class="main-header">
        <h1>🏢 PET Resource Allocation Dashboard</h1>
        <p>Comprehensive resource allocation analysis and workstream management</p>
    </div>
    """, unsafe_allow_html=True)
    
    # Load data
    with st.spinner("Loading data..."):
        df, workstream_df = load_latest_data("data")
    
    if df is None:
        st.error("No data files found. Please upload a PET Resource Allocation CSV file to the data/ folder.")
        st.info("Expected file pattern: 'PET Resource Allocation*.csv'")
        return
    
    # Sidebar filters
    st.sidebar.header("🔍 Filters")
    
    # Type filter
    if 'type_normalized' in df.columns:
        types = ['All'] + sorted(df['type_normalized'].dropna().unique().tolist())
        selected_type = st.sidebar.selectbox("Resource Type", types)
        if selected_type != 'All':
            df = df[df['type_normalized'] == selected_type]
    
    # Manager filter
    if 'manager' in df.columns:
        managers = ['All'] + sorted(df['manager'].dropna().unique().tolist())
        selected_manager = st.sidebar.selectbox("Manager", managers)
        if selected_manager != 'All':
            df = df[df['manager'] == selected_manager]
    
    # Organization filters
    if 'l3_org' in df.columns:
        l3_orgs = ['All'] + sorted(df['l3_org'].dropna().unique().tolist())
        selected_l3 = st.sidebar.selectbox("L3 Organization", l3_orgs)
        if selected_l3 != 'All':
            df = df[df['l3_org'] == selected_l3]
    
    # Main content tabs
    tab1, tab2, tab3 = st.tabs(["📊 Overview", "👥 Resources", "🎯 Workstreams"])
    
    with tab1:
        st.header("📊 Overview & KPIs")
        
        # KPI metrics
        col1, col2, col3, col4 = st.columns(4)
        
        with col1:
            total_resources = len(df)
            st.metric("Total Resources", total_resources)
        
        with col2:
            total_fte = df['calculated_total'].sum() / 100 if 'calculated_total' in df.columns else 0
            st.metric("Total FTE", f"{total_fte:.1f}")
        
        with col3:
            if 'type_normalized' in df.columns:
                employees = len(df[df['type_normalized'] == 'Employee'])
                st.metric("Employees", employees)
            else:
                st.metric("Employees", "N/A")
        
        with col4:
            if 'type_normalized' in df.columns:
                contractors = len(df[df['type_normalized'] == 'Contractor'])
                st.metric("Contractors", contractors)
            else:
                st.metric("Contractors", "N/A")
        
        # Allocation status chart
        if 'calculated_total' in df.columns:
            st.subheader("📈 Allocation Status Distribution")
            
            # Create allocation status
            df['allocation_status'] = df['calculated_total'].apply(
                lambda x: 'Overallocated' if x > 100 else 'Underallocated' if x < 100 and x > 0 else 'Unassigned'
            )
            
            status_counts = df['allocation_status'].value_counts()
            
            fig = px.pie(
                values=status_counts.values,
                names=status_counts.index,
                title="Resource Allocation Status",
                color_discrete_map={
                    'Overallocated': '#ff6b6b',
                    'Underallocated': '#ffd93d',
                    'Unassigned': '#6bcf7f'
                }
            )
            st.plotly_chart(fig, use_container_width=True)
    
    with tab2:
        st.header("👥 Resource Details")
        
        # Display main dataframe
        if not df.empty:
            # Select relevant columns for display
            display_cols = []
            for col in ['resource_name', 'manager', 'type_normalized', 'l3_org', 'vp_org', 'director_org', 'calculated_total']:
                if col in df.columns:
                    display_cols.append(col)
            
            if display_cols:
                st.dataframe(
                    df[display_cols].head(100),
                    use_container_width=True,
                    height=400
                )
            else:
                st.dataframe(df.head(100), use_container_width=True, height=400)
        
        # Download button
        if not df.empty:
            csv = df.to_csv(index=False)
            st.download_button(
                label="📥 Download Resource Data",
                data=csv,
                file_name=f"pet_resources_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.csv",
                mime="text/csv"
            )
    
    with tab3:
        st.header("🎯 Workstream Analysis")
        
        if workstream_df is not None and not workstream_df.empty:
            # Workstream FTE chart
            workstream_fte = workstream_df.groupby('workstream')['allocation_pct'].sum() / 100
            workstream_fte = workstream_fte.sort_values(ascending=False)
            
            fig = px.bar(
                x=workstream_fte.index,
                y=workstream_fte.values,
                title="FTE Allocation by Workstream",
                labels={'x': 'Workstream', 'y': 'FTE'}
            )
            fig.update_layout(xaxis_tickangle=-45)
            st.plotly_chart(fig, use_container_width=True)
            
            # Workstream details table
            st.subheader("📋 Workstream Assignments")
            display_ws_cols = ['workstream', 'resource_name', 'allocation_pct', 'manager', 'type']
            available_cols = [col for col in display_ws_cols if col in workstream_df.columns]
            
            if available_cols:
                st.dataframe(
                    workstream_df[available_cols].sort_values('allocation_pct', ascending=False),
                    use_container_width=True,
                    height=400
                )
        else:
            st.info("No workstream data available.")

    # Footer
    st.markdown("---")
    st.markdown("**PET Resource Allocation Dashboard** | Data automatically refreshed")

if __name__ == "__main__":
    main()
'''

with open('app.py', 'w') as f:
    f.write(app_code)

print("✅ Streamlit application created!")


In [ ]:
# 🚀 STEP 8: Launch Dashboard with Public Access
print("🚀 Starting PET Dashboard with ngrok tunnel...")

import subprocess
import threading
import time
from pyngrok import ngrok, conf

# Set up ngrok (you may need to sign up for a free account at https://ngrok.com)
# and set your authtoken: ngrok.set_auth_token("YOUR_TOKEN")

def run_streamlit():
    """Run Streamlit in the background"""
    subprocess.run(["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"])

# Start Streamlit in a separate thread
streamlit_thread = threading.Thread(target=run_streamlit, daemon=True)
streamlit_thread.start()

# Wait a moment for Streamlit to start
time.sleep(10)

try:
    # Create ngrok tunnel
    public_url = ngrok.connect(8501)
    print("🎉 SUCCESS! Your dashboard is now publicly available at:")
    print(f"🌐 Public URL: {public_url}")
    print(f"📱 Share this URL with your team!")
    print("")
    print("⚡ Dashboard Features:")
    print("   • Real-time resource allocation analysis")
    print("   • Interactive filters and charts")
    print("   • Workstream FTE calculations")
    print("   • Data export capabilities")
    print("")
    print("🔄 The dashboard will continue running...")
    print("⏹️ To stop: Runtime → Interrupt execution")
    
    # Keep the tunnel alive
    while True:
        time.sleep(60)
        
except Exception as e:
    print(f"❌ Error creating public tunnel: {e}")
    print("")
    print("📋 Alternative options:")
    print("1. Run locally without public access:")
    print("   !streamlit run app.py")
    print("")
    print("2. Set up ngrok account:")
    print("   - Go to: https://ngrok.com/signup")
    print("   - Get your authtoken")
    print("   - Run: !ngrok authtoken YOUR_TOKEN")
    print("   - Then re-run this cell")
    
    # Fallback: run streamlit without ngrok
    print("🔧 Starting local dashboard without public URL...")
    subprocess.run(["streamlit", "run", "app.py", "--server.port", "8501"])
